In [ ]:
# Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2
import os
import pandas as pd
from deepface import DeepFace
from datetime import datetime
from google.colab import output
from IPython.display import Javascript
from base64 import b64decode

# =========================
# STEP 1: Webcam Capture Function
# =========================
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = '📷 Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          div.appendChild(video);

          document.body.appendChild(div);

          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          video.srcObject = stream;
          await video.play();

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);

          stream.getVideoTracks()[0].stop();
          div.remove();

          return canvas.toDataURL('image/jpeg', quality);
        }
        takePhoto({{quality}});''')

    data = output.eval_js(js)
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# =========================
# STEP 2: Load Enrolled Students (jaf & dona in sih/ folder)
# =========================
folder = "sih"
if not os.path.exists(folder):
    os.makedirs(folder)

students_db = {}
for file in os.listdir(folder):
    if file.endswith(".jpg") or file.endswith(".png"):
        name = file.split(".")[0]   # filename without extension = student name
        students_db[name] = os.path.join(folder, file)

print("✅ Enrolled Students:", list(students_db.keys()))

# =========================
# STEP 3: Capture Photo from Webcam (simulate classroom)
# =========================
print("📸 Please capture classroom image...")
photo_path = take_photo("classroom.jpg")

# =========================
# STEP 4: Compare with Enrolled Students
# =========================
attendance = []
for name, path in students_db.items():
    try:
        result = DeepFace.verify(photo_path, path, enforce_detection=False)
        if result['verified']:
            attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
        else:
            attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
    except Exception as e:
        print("⚠️ Error with", name, e)

# =========================
# STEP 5: Save Attendance
# =========================
df = pd.DataFrame(attendance)
df.to_csv("attendance.csv", index=False)

print("\n✅ Attendance Recorded!\n")
print(df)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
25-09-09 03:49:03 - Directory /root/.deepface has been created
25-09-09 03:49:03 - Directory /root/.deepface/weights has been created
✅ Enrolled Students: []
📸 Please capture classroom image...


ValueError: Can't clean for JSON: <IPython.core.display.Javascript object>

In [ ]:
# Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2
import os
import pandas as pd
from deepface import DeepFace
from datetime import datetime
from google.colab import output
from base64 import b64decode

# =========================
# STEP 1: Webcam Capture Function (FIXED)
# =========================
def take_photo(filename='photo.jpg', quality=0.8):
    js = """
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📷 Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      div.appendChild(video);

      document.body.appendChild(div);

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();

      // Wait for capture
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getVideoTracks()[0].stop();
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
    takePhoto(%f);
    """ % quality

    data = output.eval_js(js)   # ✅ FIX: correctly run JS in Colab
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# =========================
# STEP 2: Load Enrolled Students (put jaf.jpg, dona.jpg inside 'sih/' folder)
# =========================
folder = "sih"
if not os.path.exists(folder):
    os.makedirs(folder)

students_db = {}
for file in os.listdir(folder):
    if file.endswith(".jpg") or file.endswith(".png"):
        name = file.split(".")[0]   # filename without extension
        students_db[name] = os.path.join(folder, file)

print("✅ Enrolled Students:", list(students_db.keys()))

# =========================
# STEP 3: Capture Classroom Photo
# =========================
print("📸 Please capture classroom image...")
photo_path = take_photo("classroom.jpg")

# =========================
# STEP 4: Face Verification
# =========================
attendance = []
for name, path in students_db.items():
    try:
        result = DeepFace.verify(photo_path, path, enforce_detection=False)
        if result['verified']:
            attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
        else:
            attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
    except Exception as e:
        print("⚠️ Error with", name, e)

# =========================
# STEP 5: Save Attendance
# =========================
df = pd.DataFrame(attendance)
df.to_csv("attendance.csv", index=False)

print("\n✅ Attendance Recorded!\n")
print(df)


✅ Enrolled Students: []
📸 Please capture classroom image...


MessageError: NotAllowedError: Permission denied

In [ ]:
# Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2
import os
import pandas as pd
from deepface import DeepFace
from datetime import datetime
from google.colab import files

# =========================
# STEP 1: Upload Classroom Image
# =========================
print("📤 Please upload a classroom image (jpg/png)")
uploaded = files.upload()

photo_path = list(uploaded.keys())[0]  # take the first uploaded file

# =========================
# STEP 2: Load Enrolled Students (sih/jaf.jpg, sih/dona.jpg)
# =========================
folder = "sih"
students_db = {}
for file in os.listdir(folder):
    if file.endswith(".jpg") or file.endswith(".png"):
        name = file.split(".")[0]
        students_db[name] = os.path.join(folder, file)

print("✅ Enrolled Students:", list(students_db.keys()))

# =========================
# STEP 3: Compare Classroom Image with Enrolled Students
# =========================
attendance = []
for name, path in students_db.items():
    try:
        result = DeepFace.verify(photo_path, path, enforce_detection=False)
        if result['verified']:
            attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
        else:
            attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
    except Exception as e:
        print("⚠️ Error with", name, e)

# =========================
# STEP 4: Save Attendance
# =========================
df = pd.DataFrame(attendance)
df.to_csv("attendance.csv", index=False)

print("\n✅ Attendance Recorded!\n")
print(df)


📤 Please upload a classroom image (jpg/png)


Saving sih (1).zip to sih (1).zip
✅ Enrolled Students: ['class']


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5


25-09-08 15:06:28 - 🔗 vgg_face_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5 to /root/.deepface/weights/vgg_face_weights.h5...


100%|██████████| 580M/580M [00:08<00:00, 68.3MB/s]


⚠️ Error with class Exception while processing img1_path

✅ Attendance Recorded!

Empty DataFrame
Columns: []
Index: []


In [ ]:
import zipfile

zip_path = "sih.zip"
extract_folder = "sih"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("✅ Extracted files:", os.listdir(extract_folder))


✅ Extracted files: ['sih.zip', '.ipynb_checkpoints', 'attendance.csv ', 'sih', 'class.jpg']


In [ ]:
# ✅ Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2
import os
import pandas as pd
from deepface import DeepFace
from datetime import datetime

# STEP 1: ENROLLMENT (Register Students with Images)
# Upload student face images in "students/" folder
# Example: students/rahul.jpg, students/anita.jpg

if not os.path.exists("students"):
    os.makedirs("students")

print("✅ Please upload student images in 'students/' folder before proceeding.")

# STEP 2: LOAD STUDENTS DATABASE
students_db = {}
for file in os.listdir("students"):
    if file.lower().endswith((".jpg", ".png")):
        name = os.path.splitext(file)[0]
        students_db[name] = os.path.join("students", file)

print("📌 Enrolled Students:", list(students_db.keys()))

# STEP 3: ATTENDANCE FUNCTION
def mark_attendance(class_image):
    # Read classroom photo
    frame = cv2.imread(class_image)

    attendance = []
    for name, path in students_db.items():
        try:
            result = DeepFace.verify(img1_path=frame, img2_path=path, enforce_detection=False, model_name="Facenet")
            if result['verified']:
                attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
            else:
                attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
        except Exception as e:
            print("⚠ Error with", name, ":", e)

    return attendance

# STEP 4: TEST WITH A SAMPLE CLASS IMAGE
test_image = "class.jpg"  # Upload a classroom photo
if os.path.exists(test_image):
    records = mark_attendance(test_image)
    df = pd.DataFrame(records)
    df.to_csv("attendance.csv", index=False)
    print("✅ Attendance Recorded!")
    print(df)
else:
    print("⚠ Please upload a classroom image named 'class.jpg' for testing.")

✅ Please upload student images in 'students/' folder before proceeding.
📌 Enrolled Students: []
⚠ Please upload a classroom image named 'class.jpg' for testing.


In [ ]:
# ✅ Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2
import os
import pandas as pd
from deepface import DeepFace
from datetime import datetime
import zipfile

# STEP 1: EXTRACT sih.zip → sih/
zip_path = "sih.zip"
extract_folder = "sih"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("✅ Extracted student images:", os.listdir(extract_folder))
else:
    print("⚠️ Please upload sih.zip with jaf.jpg and dona.jpg inside")

# STEP 2: LOAD STUDENTS DATABASE
students_db = {}
for file in os.listdir(extract_folder):
    if file.lower().endswith((".jpg", ".png")):
        name = os.path.splitext(file)[0]   # file name without extension
        students_db[name] = os.path.join(extract_folder, file)

print("📌 Enrolled Students:", list(students_db.keys()))

# STEP 3: ATTENDANCE FUNCTION
def mark_attendance(class_image):
    attendance = []
    for name, path in students_db.items():
        try:
            result = DeepFace.verify(img1_path=class_image, img2_path=path, enforce_detection=False, model_name="Facenet")
            if result['verified']:
                attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
            else:
                attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
        except Exception as e:
            print("⚠ Error with", name, ":", e)
    return attendance

# STEP 4: TEST WITH class.jpg
test_image = "class.jpg"
if os.path.exists(test_image):
    records = mark_attendance(test_image)
    df = pd.DataFrame(records)
    df.to_csv("attendance.csv", index=False)
    print("\n✅ Attendance Recorded!\n")
    print(df)
else:
    print("⚠️ Please upload a classroom image named 'class.jpg'")


✅ Extracted student images: ['sih.zip', '.ipynb_checkpoints', 'attendance.csv ', 'sih', 'class.jpg']
📌 Enrolled Students: ['class']
⚠️ Please upload a classroom image named 'class.jpg'


In [ ]:
# ✅ Install dependencies
!pip install opencv-python-headless deepface pandas openpyxl

import cv2, os, pandas as pd
from deepface import DeepFace
from datetime import datetime
import zipfile
import numpy as np

# --------------------------
# STEP 1: Upload Required Files
# --------------------------
print("📂 Please upload:")
print(" - class.jpg (classroom image)")
print(" - attendance.xlsx (Excel with Name column)")
print(" - sih.zip (contains student images: dona.jpg, jaf.jpg, etc.)")

uploaded = files.upload()

# Extract sih.zip if uploaded
if "sih.zip" in uploaded:
    import zipfile
    with zipfile.ZipFile("sih.zip", "r") as zip_ref:
        zip_ref.extractall("sih")

# --------------------------
# STEP 2: Load Attendance Excel
# --------------------------
attendance_df = pd.read_excel("attendance.xlsx")

# Initialize status and time columns
attendance_df["Status"] = "Absent"
attendance_df["Time"] = ""

print("✅ Attendance sheet loaded:")
print(attendance_df)


# --------------------------
# STEP 3: Load Students
# --------------------------
students_db = {}
# Use the extracted folder name if sih.zip was uploaded, otherwise assume 'sih'
student_image_folder = "sih" if "sih.zip" in uploaded else "sih"

if os.path.exists(student_image_folder):
    for file in os.listdir(student_image_folder):
        if file.lower().endswith((".jpg", ".png")):
            name = os.path.splitext(file)[0]
            students_db[name] = os.path.join(student_image_folder, file)
else:
    print(f"⚠️ Student image folder '{student_image_folder}' not found. Please upload sih.zip")


print("📌 Enrolled Students:", list(students_db.keys()))


# --------------------------
# STEP 4: Attendance Function (handles multiple faces)
# --------------------------
def mark_attendance(class_image):
    attendance_records = attendance_df.copy() # Start with the loaded attendance sheet

    # Load classroom image
    img = cv2.imread(class_image)
    if img is None:
        print(f"⚠️ Unable to read {class_image}")
        return attendance_records

    # Detect faces in the classroom
    try:
        faces = DeepFace.extract_faces(img_path=class_image, enforce_detection=False)
        print(f"📷 Found {len(faces)} faces in classroom image")
    except Exception as e:
        print("⚠ Error detecting faces:", e)
        faces = []

    # Keep track of matched students by name
    matched_students = set()

    for face_obj in faces:
        face_img = face_obj['face']
        # Convert face_img from numpy array to BGR image path or numpy array
        # DeepFace.verify can accept numpy arrays for img1_path
        face_img_bgr = (face_img * 255).astype(np.uint8) # Convert normalized float to uint8

        for student_name, student_path in students_db.items():
            # Avoid matching the same student multiple times
            if student_name.lower() in matched_students:
                continue

            try:
                # Use numpy array for the detected face and path for the student image
                result = DeepFace.verify(img1_path=face_img_bgr, img2_path=student_path, enforce_detection=False, model_name="Facenet")
                if result['verified']:
                    matched_students.add(student_name.lower())
                    # Update the attendance_records DataFrame
                    attendance_records.loc[attendance_records['Name'].str.lower() == student_name.lower(), 'Status'] = 'Present'
                    attendance_records.loc[attendance_records['Name'].str.lower() == student_name.lower(), 'Time'] = datetime.now().strftime("%H:%M:%S")

            except Exception as e:
                # This specific error for face detection on the student image is less critical for the overall process
                # print(f"⚠ Error verifying {student_name}:", e)
                pass # Suppress errors for individual student verification attempts

    # Mark absent students (already initialized as Absent)
    # The initial DataFrame is already set to Absent, only present students are updated.

    return attendance_records

# --------------------------
# STEP 5: Process and Save Results
# --------------------------
test_image = "class.jpg"
if os.path.exists(test_image):
    print("\nProcessing attendance...")
    final_attendance_df = mark_attendance(test_image)

    # Save the updated attendance DataFrame to Excel
    final_attendance_df.to_excel("attendance_result.xlsx", index=False)

    print("\n✅ Attendance Recorded!\n")
    print(final_attendance_df)

    # Optional: Visualize results on the image (requires successful face detection and matching)
    try:
        img_bgr = cv2.imread(test_image)
        if img_bgr is not None:
            faces = DeepFace.extract_faces(img_path=test_image, enforce_detection=False)
            for face_obj in faces:
                 x, y, w, h = face_obj['facial_area']['x'], face_obj['facial_area']['y'], face_obj['facial_area']['w'], face_obj['facial_area']['h']
                 # Attempt to match the detected face to a student to get their name
                 matched_name = "Unknown"
                 color = (0, 0, 255)  # Default color for unknown (Red)

                 face_img = (face_obj['face'] * 255).astype(np.uint8) # Convert normalized float to uint8

                 for student_name, student_path in students_db.items():
                    try:
                        result = DeepFace.verify(img1_path=face_img, img2_path=student_path, enforce_detection=False, model_name="Facenet")
                        if result['verified']:
                            matched_name = student_name
                            color = (0, 255, 0) # Green for present
                            break # Found a match, move to the next detected face
                    except Exception as e:
                         pass # Suppress errors for individual student verification attempts

                 cv2.rectangle(img_bgr, (x, y), (x+w, y+h), color, 2)
                 cv2.putText(img_bgr, matched_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            cv2.imwrite("class_with_attendance.jpg", img_bgr)
            print("\n✅ Annotated classroom image saved as class_with_attendance.jpg")

            # Show annotated classroom image
            plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.show()

            # Download results
            files.download("attendance_result.xlsx")
            files.download("class_with_attendance.jpg")

        else:
            print("⚠️ Could not load class.jpg for annotation.")

    except Exception as e:
        print(f"⚠️ Error during image annotation or download: {e}")


else:
    print("⚠️ Please upload a classroom image named 'class.jpg'")

📂 Please upload:
 - class.jpg (classroom image)
 - attendance.xlsx (Excel with Name column)
 - sih.zip (contains student images: dona.jpg, jaf.jpg, etc.)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# =============================
# Step 1: Install dependencies
# =============================
!pip install face_recognition cmake dlib openpyxl

import cv2
import face_recognition
import pandas as pd
import os
from google.colab import files
import matplotlib.pyplot as plt

# =============================
# Step 2: Upload Required Files
# =============================
print("📂 Please upload:")
print(" - class.jpg (classroom image)")
print(" - attendance.xlsx (SeatID,Name)")
print(" - sih.zip (contains student images: dona.jpg, jaf.jpg, etc.)")

uploaded = files.upload()

# If sih is uploaded as a zip → extract it
if "sih.zip" in uploaded:
    import zipfile
    with zipfile.ZipFile("sih.zip", "r") as zip_ref:
        zip_ref.extractall("sih")

# =============================
# Step 3: Load Attendance Excel
# =============================
attendance_df = pd.read_excel("attendance.xlsx")   # <-- Excel instead of CSV
print("✅ Attendance sheet loaded:")
print(attendance_df)

# =============================
# Step 4: Load Student Encodings
# =============================
student_encodings = {}
for img_name in os.listdir("sih"):
    if img_name.endswith((".jpg", ".png", ".jpeg")):
        student_name = os.path.splitext(img_name)[0]  # dona.jpg -> dona
        img_path = os.path.join("sih", img_name)

        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img)

        if len(encs) > 0:
            student_encodings[student_name] = encs[0]
        else:
            print(f"⚠️ No face found in {img_name}")

print("✅ Loaded student encodings:", list(student_encodings.keys()))

# =============================
# Step 5: Load Classroom Image
# =============================
class_img = face_recognition.load_image_file("class.jpg")
class_locations = face_recognition.face_locations(class_img)
class_encodings = face_recognition.face_encodings(class_img, class_locations)

print(f"📷 Found {len(class_locations)} faces in classroom image")

# =============================
# Step 6: Match Students
# =============================
attendance_result = []

# Default: everyone Absent
for i, row in attendance_df.iterrows():
    attendance_result.append({
        "SeatID": row["SeatID"],
        "Name": row["Name"],
        "Status": "Absent"
    })

# Compare detected faces with enrolled
for face_encoding, face_location in zip(class_encodings, class_locations):
    matches = face_recognition.compare_faces(list(student_encodings.values()), face_encoding, tolerance=0.5)
    face_distances = face_recognition.face_distance(list(student_encodings.values()), face_encoding)

    if True in matches:
        best_match_index = face_distances.argmin()
        matched_name = list(student_encodings.keys())[best_match_index]

        # Mark that student as Present
        for entry in attendance_result:
            if entry["Name"].lower() == matched_name.lower():
                entry["Status"] = "Present"

# =============================
# Step 7: Save Results
# =============================
attendance_out = pd.DataFrame(attendance_result)
attendance_out.to_excel("attendance_result.xlsx", index=False)  # <-- Excel output

# Draw results on classroom image
img_bgr = cv2.cvtColor(class_img, cv2.COLOR_RGB2BGR)

for (top, right, bottom, left), result in zip(class_locations, attendance_result):
    color = (0, 255, 0) if result["Status"] == "Present" else (0, 0, 255)
    cv2.rectangle(img_bgr, (left, top), (right, bottom), color, 2)
    cv2.putText(img_bgr, result["Name"], (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

cv2.imwrite("class_with_attendance.jpg", img_bgr)

print("✅ Attendance processed!")
print(attendance_out)

# Show annotated classroom image
plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Download results
files.download("attendance_result.xlsx")
files.download("class_with_attendance.jpg")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=b0463fbc4ec5d5fd2e8af0926f451800fe1388dd0704f5194a21f6ecff4b3e79
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/.tmpVDKkM0/sdists-v9/pypi/dlib/19.24.6/R6xBjnnUENKzsvtIcAKWo/src/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
# =============================
# Step 1: Install dependencies
# =============================
!pip install face_recognition cmake dlib openpyxl

# =============================
# Step 2: Import Libraries (force CPU mode)
# =============================
import cv2
import face_recognition
import pandas as pd
import os
from google.colab import files
import matplotlib.pyplot as plt

# Force CPU only (avoid CUDA errors)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# =============================
# Step 3: Upload Required Files
# =============================
print("📂 Please upload:")
print(" - class.jpg (classroom image)")
print(" - attendance.xlsx (Excel with SeatID, Name)")
print(" - sih.zip (folder of student images: dona.jpg, jaf.jpg, etc.)")

uploaded = files.upload()

# If sih is uploaded as a zip → extract it
if "sih.zip" in uploaded:
    import zipfile
    with zipfile.ZipFile("sih.zip", "r") as zip_ref:
        zip_ref.extractall("sih")

# =============================
# Step 4: Load Attendance Excel
# =============================
attendance_df = pd.read_excel("attendance.xlsx")   # <-- Excel input
print("✅ Attendance sheet loaded:")
print(attendance_df)

# =============================
# Step 5: Load Student Encodings
# =============================
student_encodings = {}
for img_name in os.listdir("sih"):
    if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        student_name = os.path.splitext(img_name)[0]  # dona.jpg -> dona
        img_path = os.path.join("sih", img_name)

        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img)

        if len(encs) > 0:
            student_encodings[student_name] = encs[0]
        else:
            print(f"⚠️ No face found in {img_name}")

print("✅ Loaded student encodings:", list(student_encodings.keys()))

# =============================
# Step 6: Load Classroom Image
# =============================
class_img = face_recognition.load_image_file("class.jpg")
class_locations = face_recognition.face_locations(class_img, model="hog")  # CPU-friendly
class_encodings = face_recognition.face_encodings(class_img, class_locations)

print(f"📷 Found {len(class_locations)} faces in classroom image")

# =============================
# Step 7: Match Students
# =============================
attendance_result = []

# Default: everyone Absent
for i, row in attendance_df.iterrows():
    attendance_result.append({
        "SeatID": row["SeatID"],
        "Name": row["Name"],
        "Status": "Absent"
    })

# Compare detected faces with enrolled
for face_encoding, face_location in zip(class_encodings, class_locations):
    matches = face_recognition.compare_faces(list(student_encodings.values()), face_encoding, tolerance=0.5)
    face_distances = face_recognition.face_distance(list(student_encodings.values()), face_encoding)

    if True in matches:
        best_match_index = face_distances.argmin()
        matched_name = list(student_encodings.keys())[best_match_index]

        # Mark that student as Present
        for entry in attendance_result:
            if entry["Name"].lower() == matched_name.lower():
                entry["Status"] = "Present"

# =============================
# Step 8: Save Results
# =============================
attendance_out = pd.DataFrame(attendance_result)
attendance_out.to_excel("attendance_result.xlsx", index=False)  # <-- Excel output

# Draw results on classroom image
img_bgr = cv2.cvtColor(class_img, cv2.COLOR_RGB2BGR)

for (top, right, bottom, left), encoding in zip(class_locations, class_encodings):
    # Find which student this face belongs to
    matches = face_recognition.compare_faces(list(student_encodings.values()), encoding, tolerance=0.5)
    name = "Unknown"
    color = (0, 0, 255)  # default red for unknown

    if True in matches:
        best_match_index = matches.index(True)
        name = list(student_encodings.keys())[best_match_index]
        color = (0, 255, 0)  # green for recognized

    cv2.rectangle(img_bgr, (left, top), (right, bottom), color, 2)
    cv2.putText(img_bgr, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

cv2.imwrite("class_with_attendance.jpg", img_bgr)

print("✅ Attendance processed!")
print(attendance_out)

# Show annotated classroom image
plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Download results
files.download("attendance_result.xlsx")
files.download("class_with_attendance.jpg")


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/.tmpVDKkM0/sdists-v9/pypi/dlib/19.24.6/R6xBjnnUENKzsvtIcAKWo/src/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
# =============================
# Step 1: Install dependencies
# =============================
!pip install face_recognition cmake dlib openpyxl

# =============================
# Step 2: Import Libraries
# =============================
import cv2
import face_recognition
import pandas as pd
import os
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt

# =============================
# Step 3: Upload Required Files
# =============================
print("📂 Please upload:")
print(" - class.jpg (classroom image)")
print(" - attendance.xlsx (Excel with columns: name, status, time)")
print(" - sih.zip (folder of student images: jaf.jpg, donna.jpg, etc.)")

uploaded = files.upload()

# Extract sih.zip if uploaded
if "sih.zip" in uploaded:
    import zipfile
    with zipfile.ZipFile("sih.zip", "r") as zip_ref:
        zip_ref.extractall("sih")

# =============================
# Step 4: Load Attendance Excel
# =============================
attendance_df = pd.read_excel("attendance.xlsx")

# Reset status and time (start fresh)
attendance_df["status"] = "Absent"
attendance_df["time"] = ""

print("✅ Attendance sheet loaded:")
print(attendance_df)

# =============================
# Step 5: Load Student Encodings
# =============================
student_encodings = {}
for img_name in os.listdir("sih"):
    if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        student_name = os.path.splitext(img_name)[0]  # jaf.jpg → jaf
        img_path = os.path.join("sih", img_name)

        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img, model="large")

        if len(encs) > 0:
            student_encodings[student_name.lower()] = encs[0]
        else:
            print(f"⚠️ No face found in {img_name}")

print("✅ Loaded student encodings:", list(student_encodings.keys()))

# =============================
# Step 6: Load Classroom Image
# =============================
class_img = face_recognition.load_image_file("class.jpg")
class_locations = face_recognition.face_locations(class_img, model="hog")  # CPU detector
class_encodings = face_recognition.face_encodings(class_img, class_locations, model="large")

print(f"📷 Found {len(class_locations)} faces in classroom image")

# =============================
# Step 7: Match Students
# =============================
now_time = datetime.now().strftime("%H:%M:%S")  # current time

for face_encoding in class_encodings:
    matches = face_recognition.compare_faces(list(student_encodings.values()), face_encoding, tolerance=0.5)
    face_distances = face_recognition.face_distance(list(student_encodings.values()), face_encoding)

    if True in matches:
        best_match_index = face_distances.argmin()
        matched_name = list(student_encodings.keys())[best_match_index]

        # Mark as present in Excel
        for i, row in attendance_df.iterrows():
            if row["name"].lower() == matched_name:
                attendance_df.at[i, "status"] = "Present"
                attendance_df.at[i, "time"] = now_time

# =============================
# Step 8: Save Results
# =============================
attendance_df.to_excel("attendance_result.xlsx", index=False)
print("✅ Final Attendance:")
print(attendance_df)

# Download updated Excel
files.download("attendance_result.xlsx")


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/.tmpVDKkM0/sdists-v9/pypi/dlib/19.24.6/R6xBjnnUENKzsvtIcAKWo/src/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
!pip uninstall -y face_recognition dlib
!pip install face_recognition==1.3.0 dlib==19.22.1


Found existing installation: face-recognition 1.3.0
Uninstalling face-recognition-1.3.0:
  Successfully uninstalled face-recognition-1.3.0
Found existing installation: dlib 19.24.6
Uninstalling dlib-19.24.6:
  Successfully uninstalled dlib-19.24.6
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
  Running setup.py clean for dlib
Failed to build dlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (dlib)


In [ ]:
import cv2
import face_recognition
import pandas as pd
import os
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
!pip install face_recognition==1.3.0 dlib==19.18.0 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 82.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 157.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 128.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 131.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
  Running setup.py clean for dlib
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=7ad

In [ ]:
!apt-get update
!apt-get install -y cmake
!pip install face_recognition==1.3.0 dlib==19.18.0 --force-reinstall --no-cache-dir

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,005 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,239 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,273 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://security.ubuntu.com/ubuntu jamm

In [ ]:
!pip install face_recognition

In [ ]:
# ✅ Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2, os, pandas as pd
from deepface import DeepFace
from datetime import datetime
import zipfile

# STEP 1: Extract sih.zip into sih/
zip_path = "sih.zip"
extract_folder = "sih"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("✅ Extracted student images:", os.listdir(extract_folder))
else:
    print("⚠️ Please upload sih.zip with jaf.jpg and dona.jpg inside")

# STEP 2: Load Students
students_db = {}
for file in os.listdir(extract_folder):
    if file.lower().endswith((".jpg", ".png")):
        name = os.path.splitext(file)[0]
        students_db[name] = os.path.join(extract_folder, file)

print("📌 Enrolled Students:", list(students_db.keys()))

# STEP 3: Attendance Function
def mark_attendance(class_image):
    attendance = []
    for name, path in students_db.items():
        try:
            result = DeepFace.verify(img1_path=class_image, img2_path=path, enforce_detection=False, model_name="Facenet")
            if result['verified']:
                attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
            else:
                attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
        except Exception as e:
            print("⚠ Error with", name, ":", e)
    return attendance

# STEP 4: Test with class.jpg
test_image = "class.jpg"
if os.path.exists(test_image):
    records = mark_attendance(test_image)
    df = pd.DataFrame(records)
    df.to_csv("attendance.csv", index=False)
    print("\n✅ Attendance Recorded!\n")
    print(df)
else:
    print("⚠️ Please upload a classroom image named 'class.jpg'")


✅ Extracted student images: ['sih']
📌 Enrolled Students: []

✅ Attendance Recorded!

Empty DataFrame
Columns: []
Index: []


In [6]:
# ✅ Install dependencies
!pip install opencv-python-headless deepface pandas

import cv2, os, pandas as pd
from deepface import DeepFace
from datetime import datetime
import zipfile

# STEP 1: Extract sih.zip into sih/
zip_path = "sih.zip"
extract_folder = "sih"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("✅ Extracted contents:", os.listdir(extract_folder))
else:
    print("⚠️ Please upload sih.zip with jaf.jpg and dona.jpg inside")

# STEP 2: Load Students (recursively search inside sih/)
students_db = {}
for root, dirs, files in os.walk(extract_folder):   # ✅ walks all subfolders
    for file in files:
        if file.lower().endswith((".jpg", ".png")):
            name = os.path.splitext(file)[0]
            students_db[name] = os.path.join(root, file)

print("📌 Enrolled Students:", list(students_db.keys()))

# STEP 3: Attendance Function
def mark_attendance(class_image):
    attendance = []
    for name, path in students_db.items():
        try:
            result = DeepFace.verify(img1_path=class_image, img2_path=path, enforce_detection=False, model_name="Facenet")
            if result['verified']:
                attendance.append({"Name": name, "Status": "Present", "Time": datetime.now()})
            else:
                attendance.append({"Name": name, "Status": "Absent", "Time": datetime.now()})
        except Exception as e:
            print("⚠ Error with", name, ":", e)
    return attendance

# STEP 4: Test with class.jpg
test_image = "class.jpg"
if os.path.exists(test_image):
    records = mark_attendance(test_image)
    df = pd.DataFrame(records)
    df.to_csv("attendance.csv", index=False)
    print("\n✅ Attendance Recorded!\n")
    print(df)
else:
    print("⚠️ Please upload a classroom image named 'class.jpg'")


✅ Extracted contents: ['sih']
📌 Enrolled Students: ['dona.jpg', 'jaf.jpg']

✅ Attendance Recorded!

       Name   Status                       Time
0  dona.jpg   Absent 2025-09-09 04:20:49.455583
1   jaf.jpg  Present 2025-09-09 04:20:52.921008
